In [1]:
from pathlib import Path
import sumolib
import typing as ty

import xml.etree.ElementTree as ET

import logzero
logger = logzero.logger

In [2]:
PATH_SUMO_NET = "/home/kensuke_mit/sumo-sim-monaco-scenario/simulation_extractions/sumo_configs/f1_scenario/in/most.net.xml"
assert Path(PATH_SUMO_NET).exists(), f"not found: {PATH_SUMO_NET}"

In [3]:
import sumolib

# Load the network
net = sumolib.net.readNet(PATH_SUMO_NET)  # Replace with your network file

# Initialize an empty list to store the prohibited edges
prohibited_edges = []

# Iterate over all edges in the network
for edge in net.getEdges():
    # Check if the edge allows any vehicle class
    if not any(edge.allows(vClass) for vClass in ['passenger', 'bus', 'truck', 'motorcycle', 'bicycle', 'pedestrian']):
        # If the edge does not allow any vehicle class, add it to the list
        prohibited_edges.append(edge)
        logger.debug(f"edge: {edge.getID()}")
    # end if
# end for
# Now, `prohibited_edges` contains all edges that all vehicles are prohibited to pass through


[D 240211 10:28:54 2605303808:15] edge: -152140
[D 240211 10:28:54 2605303808:15] edge: -152191
[D 240211 10:28:54 2605303808:15] edge: -152289
[D 240211 10:28:54 2605303808:15] edge: -152333
[D 240211 10:28:54 2605303808:15] edge: -152420
[D 240211 10:28:54 2605303808:15] edge: -152464
[D 240211 10:28:54 2605303808:15] edge: -152650#0
[D 240211 10:28:54 2605303808:15] edge: -152650#1
[D 240211 10:28:54 2605303808:15] edge: -152650#2
[D 240211 10:28:54 2605303808:15] edge: -152662
[D 240211 10:28:54 2605303808:15] edge: -152677#0
[D 240211 10:28:54 2605303808:15] edge: -152677#1
[D 240211 10:28:54 2605303808:15] edge: -152677#2
[D 240211 10:28:54 2605303808:15] edge: -152677#3
[D 240211 10:28:54 2605303808:15] edge: -152677#4
[D 240211 10:28:54 2605303808:15] edge: -152677#5
[D 240211 10:28:54 2605303808:15] edge: -152762
[D 240211 10:28:54 2605303808:15] edge: -152770
[D 240211 10:28:54 2605303808:15] edge: -152787#0
[D 240211 10:28:54 2605303808:15] edge: -152787#1
[D 240211 10:28:54

In [4]:
logger.debug(f'N(prohibited lanes) = {len(prohibited_edges)}')

[D 240211 10:28:58 2824815818:1] N(prohibited lanes) = 90


In [5]:
seq_prohibited_edges = [__e.getID() for __e in prohibited_edges]

# Vehicle Route Modifications

I obatined `prohibited_edges`. I search agents routes. If agents' routes contain the prohibited edges, I delete it from the pre-defined route.

In [7]:
PATH_ROUTE_DIR = "/home/kensuke_mit/sumo-sim-monaco-scenario/simulation_extractions/sumo_configs/f1_scenario/in/route"
PATH_ADD_DIR = "/home/kensuke_mit/sumo-sim-monaco-scenario/simulation_extractions/sumo_configs/f1_scenario/in/add"

In [23]:
path_route_dir = Path(PATH_ROUTE_DIR)
assert path_route_dir.exists(), f"not found: {path_route_dir}"

Bus routes

In [19]:
path_bus_route_definition = path_route_dir / "most.buses.flows.xml"
assert path_bus_route_definition.exists(), f"not found: {path_bus_route_definition}"

In [32]:
# updating the bus routing.
tree_xml = ET.parse(path_bus_route_definition.as_posix())
root = tree_xml.getroot()

for elem in root:
    # do something with each element here
    element_name: str = elem.tag
    if element_name == 'route':
        __route_attrib = elem.attrib
        __seq_edges: ty.List[str] = __route_attrib['edges'].split()
        # redundant approach to remove the prohibited edges, but I do this style to keep the route ordering.
        __seq_edges_updated = [__edge for __edge in __seq_edges if __edge not in seq_prohibited_edges]
        # logger.debug(f"Bus-No: {__route_attrib['id']}, before-mod-N(edges): {len(__seq_edges)}, updated-N(edges): {len(__seq_edges_updated)}")
        # updating edge information.
        __route_attrib['edges'] = " ".join(__seq_edges_updated)
    # end if
# end for

tree_xml.write(path_bus_route_definition.as_posix())
logger.debug(f"done: {path_bus_route_definition}")

[D 240210 20:55:26 3984522118:20] done: /home/kensuke_mit/sumo-sim-monaco-scenario/simulation_extractions/sumo_configs/f1_scenario/in/route/most.buses.flows.xml


Updating parking information, setting the max-capacity=0.
Getting the parking-id where the parking locates on the prohibited routing.

In [21]:
path_parking_add = Path(PATH_ADD_DIR) / 'most.parking.allvisible.add.xml'

tree_xml = ET.parse(path_parking_add.as_posix())
root = tree_xml.getroot()

seq_parking_id_unavailable = []

for elem in root:
    # do something with each element here
    element_name: str = elem.tag
    if element_name == 'parkingArea':
        _p_id = elem.attrib['id']
        _p_lane_id = elem.attrib['lane']
        _p_edge_id = _p_lane_id.split('_')[0]
        # logger.debug(_p_edge_id)
        # when the parking area is located on the prohibited edge, 
        if _p_edge_id in seq_prohibited_edges:
            seq_parking_id_unavailable.append(_p_id)
            # set the parking capacity to 0.
            elem.attrib['roadsideCapacity'] = "0"
            logger.debug(f"parking-id: {_p_id}, Set 0 capacity.")
        # end if
    # end if
    # updating rerouter
    if element_name == 'rerouter':
        iter_rerouter_def = elem.findall('interval')
        for __interval_def in iter_rerouter_def:
            iter_rerouter_def = __interval_def.findall('parkingAreaReroute')
            for _elem_park_reroute in iter_rerouter_def:
                _p_id = _elem_park_reroute.attrib['id']
                if _p_id in seq_parking_id_unavailable:
                    _elem_park_reroute.attrib['visible'] = "false"
                    # logger.debug(f"parking-id: {_p_id}, Set invisible.")
                # end if
            # end for
        # end for
    # end if
# end for

tree_xml.write(path_parking_add.as_posix())
logger.debug(f"done: {path_parking_add}")


[D 240211 10:41:59 719186531:21] parking-id: 1152, Set 0 capacity.
[D 240211 10:41:59 719186531:41] done: /home/kensuke_mit/sumo-sim-monaco-scenario/simulation_extractions/sumo_configs/f1_scenario/in/add/most.parking.allvisible.add.xml


updating commercial vehicle routing.

In [24]:
# updating commercial vehicle routing.
path_commercial_route = path_route_dir / "most.commercial.rou.xml"
assert path_commercial_route.exists(), f"not found: {path_commercial_route}"

In [27]:
# updating the bus routing.
tree_xml = ET.parse(path_commercial_route.as_posix())
root = tree_xml.getroot()

for elem in root:
    # do something with each element here
    element_name: str = elem.tag
    if element_name == 'vehicle':
        _v_id = elem.attrib['id']
        
        elem_route = elem.find('route')
        __route_attrib = elem_route.attrib
        
        __seq_edges: ty.List[str] = __route_attrib['edges'].split()
        # redundant approach to remove the prohibited edges, but I do this style to keep the route ordering.
        __seq_edges_updated = [__edge for __edge in __seq_edges if __edge not in seq_prohibited_edges]
        # logger.debug(f"No: {_v_id}, before-mod-N(edges): {len(__seq_edges)}, updated-N(edges): {len(__seq_edges_updated)}")
        # updating edge information.
        __route_attrib['edges'] = " ".join(__seq_edges_updated)
        
        # updating "stop" element if the parking lot is located on the prohibited edge.
        # <stop parkingArea="1151" until="45004" />
        elem_stop = elem.find('stop')
        if elem_stop is not None:
            _p_id = elem_stop.attrib['parkingArea']
            if _p_id in seq_parking_id_unavailable:
                elem.remove(elem_stop)
                logger.debug(f"removed: {elem_stop}")
    # end if
# end for

tree_xml.write(path_commercial_route.as_posix())        
logger.debug(f"done: {path_commercial_route}")

[D 240211 10:45:42 2214637464:33] done: /home/kensuke_mit/sumo-sim-monaco-scenario/simulation_extractions/sumo_configs/f1_scenario/in/route/most.commercial.rou.xml


Pedestrian routes.

In [29]:
path_pedestrial_route = path_route_dir / "most.pedestrian.rou.xml"
assert path_pedestrial_route.exists(), f"not found: {path_pedestrial_route}"

In [30]:
"""
    <person id="pedestrian_1-1-pt_7251" type="pedestrian" depart="18001">
        <walk edges="-152836#4 -152836#5 153171#1 153151 152870#2 153160 -152969#1 -152969#0" busStop="131101"/>
        <ride busStop="131086" lines="4:SaintRoman" intended="bus_4:SaintRoman.3" depart="18470.0"/>
        <walk edges="-152349 -152590#2 152590#1"/>
    </person>
"""

"""
    <vehicle id="pedestrian_1-1-veh_105_tr" type="ptw" depart="triggered" departLane="best" arrivalPos="79.85744050395286">
        <route edges="152870#0 152870#1 152870#2 152870#3 -152832#6 -152832#5 -152832#4 -152832#3 -152832#2 -152832#1 -152832#0 -152987#1 -152987#0 -152795#1 152808 152959#3 152959#4 152959#5 152779 152818#0 152818#1 152818#2 152818#3 152816#0 152816#1 152816#2 152816#3 152816#4 152816#5 -152810#2 -152810#1 -152804"/>
        <stop parkingArea="1147" until="43492"/>
    </vehicle>
"""


# updating the bus routing.
tree_xml = ET.parse(path_pedestrial_route.as_posix())
root = tree_xml.getroot()

for elem in root:
    # do something with each element here
    element_name: str = elem.tag
    # process for person tag
    if element_name == 'person':
        elem_walk_def = elem.findall('walk')
        for _elem_walk in elem_walk_def:
            _seq_edges = _elem_walk.attrib['edges'].split()
            _seq_edges_updated = [_edge for _edge in _seq_edges if _edge not in seq_prohibited_edges]
            _elem_walk.attrib['edges'] = " ".join(_seq_edges_updated)
            if len(_seq_edges_updated) < len(_seq_edges):
                logger.debug(f"pedestrian: {elem.attrib['id']}, before-mod-N(edges): {len(_seq_edges)}, updated-N(edges): {len(_seq_edges_updated)}")
        # end for
    # end if
    
    if element_name == 'vehicle':
        elem_route = elem.find('route')
        assert elem_route is not None, f"not found: route"
        __route_attrib = elem_route.attrib
        
        __seq_edges: ty.List[str] = __route_attrib['edges'].split()
        # redundant approach to remove the prohibited edges, but I do this style to keep the route ordering.
        __seq_edges_updated = [__edge for __edge in __seq_edges if __edge not in seq_prohibited_edges]
        # logger.debug(f"No: {_v_id}, before-mod-N(edges): {len(__seq_edges)}, updated-N(edges): {len(__seq_edges_updated)}")
        # updating edge information.
        __route_attrib['edges'] = " ".join(__seq_edges_updated)
        elem_route.attrib = __route_attrib
        
        # updating stop info
        elem_stop = elem.find('stop')
        if elem_stop is not None:
            _p_id = elem_stop.attrib['parkingArea']
            if _p_id in seq_parking_id_unavailable:
                elem.remove(elem_stop)
                logger.debug(f"removed: {elem_stop}")
            # end if
        # end if
# end for

tree_xml.write(path_pedestrial_route.as_posix())        
logger.debug(f"done: {path_pedestrial_route}")

[D 240211 10:52:27 521340825:55] removed: <Element 'stop' at 0x7f3685612ae0>
[D 240211 10:52:27 521340825:32] pedestrian: pedestrian_GW2-1_2780, before-mod-N(edges): 3, updated-N(edges): 2
[D 240211 10:52:27 521340825:32] pedestrian: pedestrian_1-1-pt_3431, before-mod-N(edges): 8, updated-N(edges): 7
[D 240211 10:52:27 521340825:55] removed: <Element 'stop' at 0x7f36849540e0>
[D 240211 10:52:27 521340825:32] pedestrian: pedestrian_2-1_4638, before-mod-N(edges): 2, updated-N(edges): 0
[D 240211 10:52:27 521340825:32] pedestrian: pedestrian_GW1-1_2933, before-mod-N(edges): 2, updated-N(edges): 1
[D 240211 10:52:27 521340825:32] pedestrian: pedestrian_1-1-pt_2835, before-mod-N(edges): 20, updated-N(edges): 19
[D 240211 10:52:27 521340825:32] pedestrian: pedestrian_1-1-pt_2835, before-mod-N(edges): 2, updated-N(edges): 0
[D 240211 10:52:27 521340825:32] pedestrian: pedestrian_1-1-pt_8618, before-mod-N(edges): 4, updated-N(edges): 2
[D 240211 10:52:27 521340825:32] pedestrian: pedestrian_1-

special agents

In [31]:
path_special_route = path_route_dir / "most.special.rou.xml"
assert path_special_route.exists(), f"not found: {path_special_route}"

In [32]:
# updating the bus routing.
tree_xml = ET.parse(path_special_route.as_posix())
root = tree_xml.getroot()

for elem in root:
    # do something with each element here
    element_name: str = elem.tag    
    if element_name == 'vehicle':
        elem_route = elem.find('route')
        assert elem_route is not None, f"not found: route"
        __route_attrib = elem_route.attrib
        
        __seq_edges: ty.List[str] = __route_attrib['edges'].split()
        # redundant approach to remove the prohibited edges, but I do this style to keep the route ordering.
        __seq_edges_updated = [__edge for __edge in __seq_edges if __edge not in seq_prohibited_edges]
        # logger.debug(f"No: {_v_id}, before-mod-N(edges): {len(__seq_edges)}, updated-N(edges): {len(__seq_edges_updated)}")
        # updating edge information.
        __route_attrib['edges'] = " ".join(__seq_edges_updated)
        elem_route.attrib = __route_attrib
# end for

tree_xml.write(path_special_route.as_posix())        
logger.debug(f"done: {path_special_route}")

[D 240211 10:55:24 995089233:23] done: /home/kensuke_mit/sumo-sim-monaco-scenario/simulation_extractions/sumo_configs/f1_scenario/in/route/most.special.rou.xml
